<a href="https://colab.research.google.com/github/Far-ch/Signals-and-Systems-Project/blob/main/Signal_and_Systems_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Detection algorithms #YOLOv8
pip install ultralytics opencv-python
from ultralytics import YOLO
import cv2

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Open the input video file
video_path = "input_video.mp4"
cap = cv2.VideoCapture(video_path)

# Prepare output video writer
output_path = "detected_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, 30.0, (640, 480))

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    # Process detection results
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = float(box.conf[0])               # Confidence score
            cls = int(box.cls[0])                   # Class ID
            label = model.names[cls]                # Class label

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow("Detection", frame)
    out.write(frame)  # Save the frame to output video

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release video resources
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
#Faster R-CNN
pip install torch torchvision opencv-python
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
import cv2

# Load the pretrained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode

# COCO dataset class labels
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Open the input video file
cap = cv2.VideoCapture("input_video.mp4")

# Set up video writer to save output video with detections
out = cv2.VideoWriter("fasterrcnn_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (640, 480))

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break  # Stop the loop if no frame is read (end of video)

    # Convert the frame from BGR to RGB color space for the model
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert image to tensor and add batch dimension
    image_tensor = F.to_tensor(image).unsqueeze(0)

    with torch.no_grad():
        # Perform inference using the model
        outputs = model(image_tensor)

    # Loop over each detected object
    for box, label, score in zip(outputs[0]["boxes"], outputs[0]["labels"], outputs[0]["scores"]):
        if score > 0.6:  # Filter out low-confidence detections
            x1, y1, x2, y2 = box.int().tolist()  # Get bounding box coordinates
            class_name = COCO_INSTANCE_CATEGORY_NAMES[label]  # Get class name by label index
            # Draw rectangle around detected object
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Put class name and confidence score text above the box
            cv2.putText(frame, f"{class_name} {score:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the frame with detections to the output video
    out.write(frame)
    # Display the frame in a window
    cv2.imshow("Faster R-CNN Detection", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()
